In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-226342
Azure region: westeurope
Subscription id: aa7cf8e8-d23f-4bce-a7b9-1f0b4e0ac8ee
Resource group: aml-quickstarts-226342


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "project-cluster"
vm_size = "Standard_D2_V2"

try:
    aml_compute = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size=vm_size, max_nodes=4)
    aml_compute = ComputeTarget.create(ws, cluster_name, compute_config)

aml_compute.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling({
        "--C":uniform(0.01,1),
        "--max_iter": choice(10, 20, 50, 200)
})

# Specify a Policy
policy = BanditPolicy(slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(
    source_directory='.',
    script='train.py',
    compute_target=aml_compute,
    environment=sklearn_env
)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
    run_config=src, 
    hyperparameter_sampling=ps, 
    policy=policy, 
    primary_metric_name="Accuracy", 
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
    max_total_runs=20
)

In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
from azureml.widgets import RunDetails
### YOUR CODE HERE ###

hyperdrive_run = exp.submit(config=hyperdrive_config)

RunDetails(hyperdrive_run).show()

hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_ecf06e8a-6e00-4699-9dd9-95a608b833d6
Web View: https://ml.azure.com/runs/HD_ecf06e8a-6e00-4699-9dd9-95a608b833d6?wsid=/subscriptions/aa7cf8e8-d23f-4bce-a7b9-1f0b4e0ac8ee/resourcegroups/aml-quickstarts-226342/workspaces/quick-starts-ws-226342&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2023-02-18T14:45:38.939734][GENERATOR][INFO]Trying to sample '20' jobs from the hyperparameter space
[2023-02-18T14:45:40.0875033Z][SCHEDULER][INFO]Scheduling job, id='HD_ecf06e8a-6e00-4699-9dd9-95a608b833d6_0' 
[2023-02-18T14:45:40.2274122Z][SCHEDULER][INFO]Scheduling job, id='HD_ecf06e8a-6e00-4699-9dd9-95a608b833d6_1' 
[2023-02-18T14:45:40.2846454Z][SCHEDULER][INFO]Scheduling job, id='HD_ecf06e8a-6e00-4699-9dd9-95a608b833d6_2' 
[2023-02-18T14:45:40.3973129Z][SCHEDULER][INFO]Scheduling job, id='HD_ecf06e8a-6e00-4699-9dd9-95a608b833d6_3' 
[2023-02-18T14:45:40.4885992Z][SCHEDULER][INFO]Scheduling job, id='HD_ecf06e8a-6e00-4699-9dd9-95a608b833d6_4' 
[2023-02-1

{'runId': 'HD_ecf06e8a-6e00-4699-9dd9-95a608b833d6',
 'target': 'project-cluster',
 'status': 'Completed',
 'startTimeUtc': '2023-02-18T14:45:38.252745Z',
 'endTimeUtc': '2023-02-18T14:56:30.176836Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '18cbe29e-f4fd-43ae-8a1a-b295734e5c81',
  'user_agent': 'python/3.8.10 (Linux-5.15.0-1031-azure-x86_64-with-glibc2.17) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.48.0',
  'space_size': 'infinite_space_size',
  'score': '0.9122359796067007',
  'best_child_run_id': 'HD_ecf06e8a-6e00-4699-9dd9-95a608b833d6_16',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_ecf06e8a-6e00-4699-9dd9-95a608b833d6_16'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'teleme

In [5]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details() ['runDefinition']['arguments']

print(f"Metrics: {best_run_metrics}\nParameters: {parameter_values}")

best_run.download_files("./training/best_model.joblib", "best_model.joblib")


Metrics: {'Regularization Strength:': 0.21077994157876576, 'Max iterations:': 200, 'Accuracy': 0.9122359796067007}
Parameters: ['--C', '0.21077994157876576', '--max_iter', '200']


In [6]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
# Create TabularDataset using TabularDatasetFactory
ds = TabularDatasetFactory.from_delimited_files("https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")


In [7]:
from train import clean_data
import pandas as pd

# Use the clean_data function to clean your data.
training_data = pd.concat(clean_data(ds), axis='columns')


In [8]:
# Fix automl config
# conda activate azureml_py38
# pip install certifi==2022.9.24
# sudo chmod 755 /anaconda/envs/azureml_py38/lib/python3.8/site-packages/nimbusml/internal/libs/pybridge.so

In [24]:
### # Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.


from azureml.train.automl import AutoMLConfig



# We might use one dataframe here which contains features and labels since the X and y parameters will soon be deprecated. But since the clean_data puts out x and y we use these parameters.
# Eventually clean_data should return one dataframe
automl_config = AutoMLConfig(
    task="classification",
    training_data=training_data,
    label_column_name="y",
    iterations=10,
    iteration_timeout_minutes=2,
    primary_metric="accuracy",
    n_cross_validations=3,
    experiment_timeout_minutes=30,
)

In [ ]:
# Submit your automl run

### YOUR CODE HERE ###
automl_run = exp.submit(config=automl_config, show_output=True)


No run_configuration provided, running on local with default configuration
Running in the active local environment.


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_c772ffb5-bf7a-48ae-ab51-a1715e71c84a,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.


2023/02/18 15:48:37 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!



********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+------------------------------+--------------------------------+--------------------------------------+
|Size of the smallest class    |Name/Label of the smallest class|Number of samples in the training data|
+==============================+================================+======================================+
|3692                          |1                               |32950                                 |
+------------------------------+--------------------------------+-----

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
_, model = automl_run.get_output()
joblib.dump(model, 'automl_model')

In [ ]:

aml_compute.delete()